In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

#### 한번에 실행하는 코드


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/data/(김시연15)_2023_04_26_30_807,697.xlsx')

# '모바일'과 '웹'만 남기기xlsx
df = df[df['예매/취소방식명(관리시스템)'].isin(['모바일', '웹'])]

# # 2인 것을 버리고 새로운 데이터프레임 생성
# df = df[df['예매/취소구분'] != 2]

# '장르명' 열에서 '뮤지컬'과 '연극'만 남기기
df = df[df['장르명'].isin(['뮤지컬', '연극'])]

# '아동공연 여부' 열에서 'Y'를 제외
df = df[df['아동공연 여부'] != 'Y']

# 조건에 따른 '극장규모' 파생변수 생성
def theater_scale(row):
    if row['장르명'] == '연극':
        if row['좌석수'] < 100:
            return 0  # 소극장
        elif row['좌석수'] < 300:
            return 1  # 중극장
        else:
            return 2  # 대극장
    elif row['장르명'] == '뮤지컬':
        if row['좌석수'] < 300:
            return 0  # 소극장
        elif row['좌석수'] < 1000:
            return 1  # 중극장
        else:
            return 2  # 대극장

df['극장규모'] = df.apply(theater_scale, axis=1)

# '재관람', '매니아', '마니아' 단어가 포함된 경우 1, 아니면 0을 설정하는 함수
def check_keywords(value):
    if pd.isna(value):
        return 0
    keywords = ['재관람', '매니아', '마니아']
    for keyword in keywords:
        if keyword in value:
            return 1
    return 0

# '재관람' 컬럼 생성 및 값 설정
df['재관람'] = df['할인종류명(전송처)'].apply(lambda x: 1 if pd.notna(x) and any(keyword in x for keyword in ['재관람', '매니아', '마니아']) else 0)

# 파일 다시 저장
df.to_excel('/content/drive/MyDrive/data/(김시연15)_2023_04_26_30_807,697.xlsx')

df.head()

,좌석수,공연코드,공연일시,예매/취소일시,예매/취소구분,예매/취소매수,예매/취소금액,예매/취소방식명(관리시스템),할인금액,할인종류명(전송처),...,공연지역명,출연진내용,아동공연 여부,내한공연 여부,오픈런 여부,판매시작일시,판매종료일시,좌석등급,극장규모,재관람
0,962,PF435538,2023-04-26 10:00,2023-04-24 15:17,1,1,28000,웹,7000,단체티켓,...,경상도,NaN,N,N,N,2023/02/13 13:00:00,2023/04/28 23:59:00,"1층석(35000), 2층석(25000)",1,0
1,962,PF435538,2023-04-26 10:00,2023-04-24 15:17,1,1,28000,웹,7000,단체티켓,...,경상도,NaN,N,N,N,2023/02/13 13:00:00,2023/04/28 23:59:00,"1층석(35000), 2층석(25000)",1,0
2,962,PF435538,2023-04-26 10:00,2023-04-24 15:17,1,1,28000,웹,7000,단체티켓,...,경상도,NaN,N,N,N,2023/02/13 13:00:00,2023/04/28 23:59:00,"1층석(35000), 2층석(25000)",1,0
3,962,PF435538,2023-04-26 10:00,2023-04-24 15:17,1,1,28000,웹,7000,단체티켓,...,경상도,NaN,N,N,N,2023/02/13 13:00:00,2023/04/28 23:59:00,"1층석(35000), 2층석(25000)",1,0
4,962,PF435538,2023-04-26 10:00,2023-04-24 15:17,1,1,28000,웹,7000,단체티켓,...,경상도,NaN,N,N,N,2023/02/13 13:00:00,2023/04/28 23:59:00,"1층석(35000), 2층석(25000)",1,0


#### 행 지우기 작업

In [ ]:
# '예매/취소방식명(관리시스템)' 열의 unique 값 확인
unique_values_system = df['예매/취소방식명(관리시스템)'].unique()
print("Unique values in '예매/취소방식명(관리시스템)':", unique_values_system)

Unique values in '예매/취소방식명(관리시스템)': ['전화' '모바일' '웹' '기타' '현장']


In [ ]:
# '모바일'과 '웹'만 남기기
df = df[df['예매/취소방식명(관리시스템)'].isin(['모바일', '웹'])]

In [ ]:
# 2인 것을 버리기 전에 개수를 확인
count_to_drop = df[df['예매/취소구분'] == 2].shape[0]
print(f"버려질 행의 개수: {count_to_drop}")

버려질 행의 개수: 99614


In [ ]:
# 2인 것을 버리고 새로운 데이터프레임 생성
df = df[df['예매/취소구분'] != 2]

In [ ]:
unique_values_jangre = df['장르명'].unique()
unique_values_jangre

array(['뮤지컬', '서커스/마술', '복합', '연극', '대중음악', '한국음악(국악)', '서양음악(클래식)',
       '무용(서양/한국무용)'], dtype=object)

In [ ]:
# '장르명' 열에서 '뮤지컬'과 '연극'만 남기기
df = df[df['장르명'].isin(['뮤지컬', '연극'])]

In [ ]:
unique_values_teen = df['아동공연 여부'].unique()
unique_values_teen

array(['Y', 'N'], dtype=object)

In [ ]:
# '아동공연 여부' 열에서 'Y'를 제외
df = df[df['아동공연 여부'] != 'Y']

In [ ]:
# '예매 취소 구분' 열의 unique 값 확인
unique_values_cancel = df['예매/취소구분'].unique()
print("Unique values in '예매/취소구분':", unique_values_cancel)

Unique values in '예매/취소구분': [1]


In [ ]:
# 최종 데이터프레임 확인
df

,좌석수,공연코드,공연일시,예매/취소일시,예매/취소구분,예매/취소매수,예매/취소금액,예매/취소방식명(관리시스템),할인금액,할인종류명(전송처),...,공연종료일자,장르명,공연지역명,출연진내용,아동공연 여부,내한공연 여부,오픈런 여부,판매시작일시,판매종료일시,좌석등급
7457,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,2023-07-30,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000)
7458,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,2023-07-30,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000)
7459,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,2023-07-30,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000)
7460,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,2023-07-30,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000)
7461,14,PF304959,2022-05-06 11:30,2022-04-20 16:14,1,1,69000,모바일,0,기타,...,2023-07-30,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571187,600,PF308191,2022-05-10 20:00,2022-05-10 12:52,1,1,19000,모바일,3000,기타,...,2022-05-22,뮤지컬,서울,"강혜인, 김이후, 최지혜, 임준혁, 배나라, 윤소호, 김종구 등",N,N,N,2022/05/09 14:00:00,2022/05/10 17:00:00,"R석(88000), S석(66000), A석(44000)"
571188,197,PF352013,2022-05-10 20:00,2022-05-10 14:34,1,1,9900,모바일,0,N,...,2023-05-31,연극,서울,"임정균, 강태우, 문선화, 김민서, 김락현, 이제성, 김해율 등",N,N,Y,2022/04/22 00:00:00,2022/05/10 19:00:00,전석(35000)
571189,197,PF352013,2022-05-10 20:00,2022-05-10 14:34,1,1,9900,모바일,0,N,...,2023-05-31,연극,서울,"임정균, 강태우, 문선화, 김민서, 김락현, 이제성, 김해율 등",N,N,Y,2022/04/22 00:00:00,2022/05/10 19:00:00,전석(35000)
571190,83,PF383410,2022-05-10 20:00,2022-05-10 18:37,1,1,14000,모바일,0,N,...,2023-05-31,연극,서울,"오승현, 김현서, 호천, 이정은, 지나경, 조민솔, 송대명 등",N,N,Y,2022/04/12 00:00:00,2022/05/10 19:40:00,전석(35000)


#### 파생변수 생성

##### 극장규모 열 생성

In [ ]:
# 조건에 따른 '극장규모' 파생변수 생성
def theater_scale(row):
    if row['장르명'] == '연극':
        if row['좌석수'] < 100:
            return 0  # 소극장
        elif row['좌석수'] < 300:
            return 1  # 중극장
        else:
            return 2  # 대극장
    elif row['장르명'] == '뮤지컬':
        if row['좌석수'] < 300:
            return 0  # 소극장
        elif row['좌석수'] < 1000:
            return 1  # 중극장
        else:
            return 2  # 대극장

df['극장규모'] = df.apply(theater_scale, axis=1)

# 결과 출력
df

,좌석수,공연코드,공연일시,예매/취소일시,예매/취소구분,예매/취소매수,예매/취소금액,예매/취소방식명(관리시스템),할인금액,할인종류명(전송처),...,장르명,공연지역명,출연진내용,아동공연 여부,내한공연 여부,오픈런 여부,판매시작일시,판매종료일시,좌석등급,극장규모
7457,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0
7458,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0
7459,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0
7460,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0
7461,14,PF304959,2022-05-06 11:30,2022-04-20 16:14,1,1,69000,모바일,0,기타,...,연극,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571187,600,PF308191,2022-05-10 20:00,2022-05-10 12:52,1,1,19000,모바일,3000,기타,...,뮤지컬,서울,"강혜인, 김이후, 최지혜, 임준혁, 배나라, 윤소호, 김종구 등",N,N,N,2022/05/09 14:00:00,2022/05/10 17:00:00,"R석(88000), S석(66000), A석(44000)",1
571188,197,PF352013,2022-05-10 20:00,2022-05-10 14:34,1,1,9900,모바일,0,N,...,연극,서울,"임정균, 강태우, 문선화, 김민서, 김락현, 이제성, 김해율 등",N,N,Y,2022/04/22 00:00:00,2022/05/10 19:00:00,전석(35000),1
571189,197,PF352013,2022-05-10 20:00,2022-05-10 14:34,1,1,9900,모바일,0,N,...,연극,서울,"임정균, 강태우, 문선화, 김민서, 김락현, 이제성, 김해율 등",N,N,Y,2022/04/22 00:00:00,2022/05/10 19:00:00,전석(35000),1
571190,83,PF383410,2022-05-10 20:00,2022-05-10 18:37,1,1,14000,모바일,0,N,...,연극,서울,"오승현, 김현서, 호천, 이정은, 지나경, 조민솔, 송대명 등",N,N,Y,2022/04/12 00:00:00,2022/05/10 19:40:00,전석(35000),0


In [ ]:
df['극장규모'].value_counts()

,count
극장규모,
1,52575
2,47904
0,9918


##### 재관람 열

In [ ]:
# '재관람', '매니아', '마니아' 단어가 포함된 경우 1, 아니면 0을 설정하는 함수
def check_keywords(value):
    if pd.isna(value):
        return 0
    keywords = ['재관람', '매니아', '마니아']
    for keyword in keywords:
        if keyword in value:
            return 1
    return 0

# '재관람' 컬럼 생성 및 값 설정
df['재관람'] = df['할인종류명(전송처)'].apply(lambda x: 1 if pd.notna(x) and any(keyword in x for keyword in ['재관람', '매니아', '마니아']) else 0)
df

,좌석수,공연코드,공연일시,예매/취소일시,예매/취소구분,예매/취소매수,예매/취소금액,예매/취소방식명(관리시스템),할인금액,할인종류명(전송처),...,공연지역명,출연진내용,아동공연 여부,내한공연 여부,오픈런 여부,판매시작일시,판매종료일시,좌석등급,극장규모,재관람
7457,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0,0
7458,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0,0
7459,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0,0
7460,14,PF304959,2022-05-06 11:30,2022-04-11 23:56,1,1,69000,모바일,0,기타,...,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0,0
7461,14,PF304959,2022-05-06 11:30,2022-04-20 16:14,1,1,69000,모바일,0,기타,...,제주도,NaN,N,N,Y,2021/11/01 00:00:00,9999/12/31 12:00:00,전석(69000),0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571187,600,PF308191,2022-05-10 20:00,2022-05-10 12:52,1,1,19000,모바일,3000,기타,...,서울,"강혜인, 김이후, 최지혜, 임준혁, 배나라, 윤소호, 김종구 등",N,N,N,2022/05/09 14:00:00,2022/05/10 17:00:00,"R석(88000), S석(66000), A석(44000)",1,0
571188,197,PF352013,2022-05-10 20:00,2022-05-10 14:34,1,1,9900,모바일,0,N,...,서울,"임정균, 강태우, 문선화, 김민서, 김락현, 이제성, 김해율 등",N,N,Y,2022/04/22 00:00:00,2022/05/10 19:00:00,전석(35000),1,0
571189,197,PF352013,2022-05-10 20:00,2022-05-10 14:34,1,1,9900,모바일,0,N,...,서울,"임정균, 강태우, 문선화, 김민서, 김락현, 이제성, 김해율 등",N,N,Y,2022/04/22 00:00:00,2022/05/10 19:00:00,전석(35000),1,0
571190,83,PF383410,2022-05-10 20:00,2022-05-10 18:37,1,1,14000,모바일,0,N,...,서울,"오승현, 김현서, 호천, 이정은, 지나경, 조민솔, 송대명 등",N,N,Y,2022/04/12 00:00:00,2022/05/10 19:40:00,전석(35000),0,0


In [ ]:
df['재관람'].value_counts()

,count
재관람,
0,110144
1,253


#### 파일 다시 저장하기

In [ ]:
df.to_excel('/content/drive/MyDrive/전처리/(박채연)_19년 하반기~ _22년 하반기 공모전 raw데이터_2022_05_06_10_571,192.xlsx')